# 데이터 전처리

In [14]:
from selenium import webdriver
import time
import pickle

In [21]:
URL = 'https://www.jobkorea.co.kr/starter/passassay/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=chrome_options)
driver.implicitly_wait(2)

driver.get(url=URL)

<ipython-input-21-7da07ceafcb5>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path='chromedriver', chrome_options=chrome_options)


In [16]:
def get_pageNum():
    pageNumList = driver.find_element_by_class_name('tplPagination')
    pageNumList = pageNumList.find_elements_by_tag_name('a')
    pageNumList = [url.get_attribute('href') for url in pageNumList]
    
    return pageNumList

In [17]:
def get_resumeUrlList():
    resumeList_class = driver.find_element_by_class_name('selfLists')
    resumeClass_tit = resumeList_class.find_elements_by_class_name('tit')
    corpName = [url.find_element_by_class_name('titTx').get_attribute('textContent').strip() for url in resumeClass_tit]
    resumeUrlList = [url.find_element_by_tag_name('a').get_attribute('href') for url in resumeClass_tit]
    
    return corpName, resumeUrlList

In [18]:
def get_resumeData():
    queList, ansList, adivceList = [], [], []
    
    qnaList_class = driver.find_element_by_class_name('qnaLists') 
    queList_dt = qnaList_class.find_elements_by_tag_name('dt')
    ansList_dd = qnaList_class.find_elements_by_tag_name('dd')

    queList = [q.find_element_by_class_name('tx').get_attribute('textContent').strip() for q in queList_dt]
    ansList = [a.find_element_by_class_name('tx').get_attribute('textContent').strip() for a in ansList_dd]
    
    for dd in ansList_dd:
        try:
            advs = dd.find_element_by_class_name('advice').find_elements_by_tag_name('p')
            advText = ''
            for adv in advs:
                advText += adv.get_attribute('textContent').strip() + '\n'
                
            adivceList.append(advText)
        except:
            adivceList.append('')
    
    return (queList, ansList, adivceList)

In [ ]:
link = "https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page="
# AllResumeData = {}

pageUrlList = [link + str(i) for i in range(241, 340)]
count = 4800
flag = True

error = []

for i, pageUrl in enumerate(pageUrlList):
    print("현재 URL: ", pageUrl)
    print("진행정도: {:.2f}% ({}/{})".format((i+1) / len(pageUrlList) * 100, i+1, len(pageUrlList)), end=' / ')
    driver.get(pageUrl)
    
    corpNames, resumeUrlList = get_resumeUrlList()
    print("현재 page 자소서 개수: ", len(resumeUrlList))
    for corpName, resumeURL in zip(corpNames, resumeUrlList):
        try:
            count += 1
            print("진행 상황: {}/{}".format(count, 6630), end='\r',flush=True)
            driver.get(resumeURL)

            AllResumeData[count] = (corpName, get_resumeData())
            
        except Exception as e:
            print(e)
            error.append(count)
            
    
    if count % 1000 == 0:
        f = open('jobkor_backup_{}.pickle'.format(count), 'wb')
        pickle.dump(AllResumeData, f)
        f.close()

현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page=241
진행정도: 1.01% (1/99) / 현재 page 자소서 개수:  20
현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page=242
진행정도: 2.02% (2/99) / 현재 page 자소서 개수:  20
현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page=243
진행정도: 3.03% (3/99) / 현재 page 자소서 개수:  20
현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page=244
진행정도: 4.04% (4/99) / 현재 page 자소서 개수:  20
현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&Page=245
진행정도: 5.05% (5/99) / 현재 page 자소서 개수:  20
현재 URL:  https://www.jobkorea.co.kr/starter/PassAssay?FavorCo_Stat=0&Pass_An_Stat=0&OrderBy=0&EduType=0&WorkType=0&isSaved=0&

In [24]:
f = open('jobkor_backup_{}.pickle'.format(count), 'wb')
pickle.dump(AllResumeData, f)
f.close()